<a href="https://colab.research.google.com/github/orodriguezq/orodriguezq-Senales_Y_Sistemas/blob/main/Parcial_2_SyS_2025-1/Correccion_Punto1_Parcial2_SyS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🔩 Dashboard Interactivo para Sistemas de Segundo Orden

**Punto 1 Parcial 2 SyS**  
**Omar Andrés Rodriguez Quiceno**  

---

Este notebook está diseñado para estudiar y comparar dos modelos equivalentes de sistemas de segundo orden:

1. **Sistema mecánico masa–resorte–amortiguador**  
2. **Circuito eléctrico R–L–C**

A través de una aplicación interactiva desarrollada con **Streamlit**, el usuario podrá:

- Seleccionar el tipo de respuesta del sistema:
  - Subamortiguada  
  - Sobreamortiguada  
  - Critica  
  - Inestable  
- Ajustar dinámicamente:
  - El factor de amortiguamiento \(\zeta\)  
  - La frecuencia natural \(\omega_n\)  
- Visualizar en **lazo abierto** y **lazo cerrado**:
  - Diagrama de Bode  
  - Diagrama de polos y ceros  
  - Respuestas al impulso, escalón y rampa  
  - Parámetros temporales:
    - Tiempo de levantamiento  
    - Sobreimpulso máximo  
    - Tiempo de sobreimpulso  
    - Tiempo de establecimiento  
  - Valores calculados de los componentes de cada modelo:
    - Masa \(m\), resorte \(k\) y amortiguador \(c\)  
    - Inductancia \(L\), capacitancia \(C\) y resistencia \(R\)  

---




## Instalación de Dependencias y Configuración del Entorno

En este bloque configuramos todo lo necesario para ejecutar nuestra aplicación Streamlit en Google Colab:

1. Instalamos **Streamlit**, la librería principal para crear el dashboard.  
2. Verificamos la ruta y versión de Streamlit para asegurarnos de que la instalación fue correcta.  
3. Instalamos **Librosa**, necesaria si más adelante queremos procesar audio.  
4. Descargamos y configuramos **Cloudflared**, que nos permitirá exponer nuestro servidor local de Streamlit a Internet.


In [ ]:
# ── Instalación de Streamlit ───────────────────────────────────────────────
!pip install streamlit              # Instala Streamlit para crear la interfaz web
!which streamlit                    # Muestra la ruta donde se ha instalado el ejecutable
!streamlit --version                # Comprueba la versión de Streamlit instalada

# ── Instalación de Librosa ────────────────────────────────────────────────
!pip install librosa --quiet        # Instala Librosa para procesamiento de audio (modo silencioso)

# ── Descarga y configuración de Cloudflared ─────────────────────────────────
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
                                     # Descarga la última versión de cloudflared para Linux
!chmod +x cloudflared                # Asigna permisos de ejecución al binario descargado
!mv cloudflared /usr/local/bin/      # Mueve el binario a una ubicación accesible globalmente
!pip install streamlit control matplotlib numpy
!apt-get install -y ffmpeg  # Solo si necesitas gráficos de audio


/usr/local/bin/streamlit
Streamlit, version 1.46.1
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


## Script Principal: `0_🔩_Simulacion_Sistemas.py`

En este bloque creamos el archivo que inicia la aplicación Streamlit, definiendo tanto la **configuración** de la página como la **estructura inicial** del dashboard. Aquí se establecen:

- El título y el ícono de la aplicación.  
- Un mensaje de bienvenida en la barra lateral.  
- Una descripción general de las funcionalidades principales del panel interactivo.  


## Página de Simulación: Sistema Masa–Resorte–Amortiguador

En esta sección definimos la pestaña dedicada al modelo mecánico y su equivalente eléctrico. El usuario podrá:

- **Elegir el tipo de comportamiento** del sistema:  
  - Subamortiguado  
  - Sobreamortiguado  
  - Críticamente amortiguado  
  - Inestable  
- **Ajustar** mediante sliders:  
  - El factor de amortiguamiento \(\zeta\)  
  - La frecuencia natural \(\omega_n\)  
- **Visualizar** en lazo abierto:  
  - Polos y ceros  
  - Parámetros físicos \(m, c, k\) y equivalentes eléctricos \(L, R, C\)  
  - Respuestas al impulso, escalón y rampa  
  - Diagrama de Bode  
- **Obtener** lazo cerrado automáticamente y ver:  
  - Polos del sistema en lazo cerrado  
  - Respuestas temporales (impulso, escalón, rampa)  
  - Parámetros temporales (\(M_p\), \(T_r\), \(T_p\), \(T_s\)) si es subamortiguado  

Cada paso está comentado para explicar su propósito en la generación del dashboard interactivo.


In [ ]:
%%writefile 0_🔩_Simulacion_Sistemas.py
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
import control

# --- Configuración de la página ---
st.set_page_config(
    page_title="Simulación de Sistemas",
    page_icon="🔩",
)

# --- Título principal y bienvenida ---
st.write("# 🔩 Sistemas de Segundo Orden")
st.sidebar.success("Elige una simulación en el menú lateral.")

# --- Introducción y descripción del dashboard ---
st.markdown(
    """
    ## Panel interactivo para la simulación de modelos masa-resorte-amortiguador y circuito RLC serie.
    - **Selecciona el tipo de sistema y comportamiento dinámico (subamortiguado, sobreamortiguado, crítico, inestable).**
    - **Ajusta parámetros clave (ζ y ωₙ) y observa en tiempo real las gráficas y métricas principales.**
    - **Compara respuestas y valores de componentes (m, c, k) y (L, R, C).**
    """
)

# --- Funciones auxiliares ---
def calc_mechanical_params(zeta, wn):
    m = 1.0
    k = wn**2 * m
    c = 2 * zeta * wn * m
    return m, c, k

def calc_electric_params(zeta, wn):
    C = 1.0
    L = 1 / (wn**2 * C)
    R = 2 * zeta * wn * L
    return L, R, C

# --- Sidebar de configuración ---
st.sidebar.header("Configuración")

sistema = st.sidebar.selectbox("Selecciona el sistema:", ["Masa–resorte–amortiguador", "Circuito RLC serie"])
tipo = st.sidebar.selectbox("Tipo de respuesta:", ["Subamortiguado", "Sobreamortiguado", "Crítico", "Inestable"])
wn = st.sidebar.slider("Frecuencia natural ωₙ (rad/s)", 0.1, 10.0, 2.0, 0.1)

if tipo == "Subamortiguado":
    zeta = st.sidebar.slider("ζ (0 < ζ < 1)", 0.01, 0.99, 0.5, 0.01)
elif tipo == "Sobreamortiguado":
    zeta = st.sidebar.slider("ζ (ζ > 1)", 1.01, 3.0, 1.2, 0.01)
elif tipo == "Crítico":
    zeta = 1.0
    st.sidebar.markdown("ζ = 1 (crítico)")
else:
    zeta = st.sidebar.slider("ζ (ζ < 0)", -1.0, -0.01, -0.2, 0.01)

# --- Parámetros y función de transferencia ---
if sistema == "Masa–resorte–amortiguador":
    m, c, k = calc_mechanical_params(zeta, wn)
    num = [1]
    den = [m, c, k]
    G = control.tf(num, den)
    st.write(f"**Parámetros estimados:**  m = {m:.2f} kg,  c = {c:.2f} Ns/m,  k = {k:.2f} N/m")
else:
    L, R, C = calc_electric_params(zeta, wn)
    num = [1]
    den = [L*C, R*C, 1]
    G = control.tf(num, den)
    st.write(f"**Parámetros estimados:**  L = {L:.2f} H,  R = {R:.2f} Ω,  C = {C:.2f} F")

st.write("## Función de transferencia (lazo abierto)")
st.latex(f"\\frac{{1}}{{{den[0]:.2f}s^2 + {den[1]:.2f}s + {den[2]:.2f}}}")

# --- Respuestas en el tiempo ---
t = np.linspace(0, 10, 1000)
t_imp, y_imp = control.impulse_response(G, T=t)
t_step, y_step = control.step_response(G, T=t)
t_ramp, y_ramp = control.forced_response(G, T=t, U=t)

st.subheader("Respuestas en el tiempo")
fig, axs = plt.subplots(3,1, figsize=(6,8))
axs[0].plot(t_imp, y_imp); axs[0].set_title("Respuesta al impulso")
axs[1].plot(t_step, y_step); axs[1].set_title("Respuesta al escalón")
axs[2].plot(t_ramp, y_ramp); axs[2].set_title("Respuesta a la rampa")
plt.tight_layout()
st.pyplot(fig)

# --- Polos y ceros ---
st.subheader("Diagrama de polos y ceros")
poles = np.roots(den)
zeros = np.roots(num)
fig2, ax2 = plt.subplots()
ax2.scatter(np.real(zeros), np.imag(zeros), marker='o', label='Ceros', color='blue')
ax2.scatter(np.real(poles), np.imag(poles), marker='x', label='Polos', color='red')
ax2.axhline(0, color='gray', lw=1)
ax2.axvline(0, color='gray', lw=1)
ax2.legend(); ax2.set_xlabel("Parte real"); ax2.set_ylabel("Parte imaginaria")
ax2.set_title("Plano de polos y ceros")
st.pyplot(fig2)

# --- Diagrama de Bode (módulo y fase) ---
st.subheader("Diagrama de Bode (módulo y fase)")
w = np.logspace(-2, 2, 500)
_, y, _ = control.freqresp(G, w)
mag = 20 * np.log10(np.abs(y).flatten())
phase = np.angle(y, deg=True).flatten()

fig3, (ax3, ax4) = plt.subplots(2, 1, figsize=(7,5))
ax3.semilogx(w, mag)
ax3.set_ylabel("dB"); ax3.set_title("Bode Magnitud")
ax4.semilogx(w, phase)
ax4.set_ylabel("°"); ax4.set_xlabel("rad/s"); ax4.set_title("Bode Fase")
plt.tight_layout()
st.pyplot(fig3)

# --- Métricas temporales (respuesta al escalón) ---
st.subheader("Métricas temporales (respuesta al escalón)")
t_peak = t[np.argmax(y_step)]
Mp = (np.max(y_step) - 1)*100
rise_idxs = np.where((y_step >= 0.1) & (y_step <= 0.9))[0]
tr = t[rise_idxs[-1]] - t[rise_idxs[0]] if rise_idxs.size>1 else None
settle_idxs = np.where(np.abs(y_step - 1) <= 0.02)[0]
Ts = t[settle_idxs[-1]] if settle_idxs.size>0 else None

st.write(f"**Tiempo de subida (10-90%)**: {tr:.3f} s" if tr else "**Tiempo de subida (10-90%)**: N/A")
st.write(f"**Sobreimpulso máximo**: {Mp:.2f}% en t = {t_peak:.3f} s")
st.write(f"**Tiempo de establecimiento (2%)**: {Ts:.3f} s" if Ts else "**Tiempo de establecimiento (2%)**: N/A")


Writing 0_🔩_Simulacion_Sistemas.py


## Ejecución de la Aplicación y Exposición Pública

En este bloque lanzamos la aplicación Streamlit en segundo plano y configuramos un túnel público con Cloudflared para acceder al dashboard desde cualquier lugar. Se realizan los siguientes pasos:

1. **Cierre de instancias previas** de Streamlit y Cloudflared para evitar conflictos.  
2. **Ejecución** del script principal de la aplicación en background.  
3. **Pausa** breve para asegurar que el servidor local esté activo.  
4. **Arranque** del túnel Cloudflared que redirige el puerto local 8501 a una URL accesible públicamente.  
5. **Lectura** del log de Cloudflared para extraer y mostrar la URL pública resultante.


In [ ]:
# 🟢 Ejecuta la app y crea el túnel público

import os
import time
import re

# ── Cerrar procesos antiguos de Streamlit y Cloudflared ────────────────
!pkill -f streamlit                 # Finaliza cualquier instancia previa de Streamlit
!pkill -f cloudflared               # Finaliza cualquier túnel anterior de Cloudflared

# ── Iniciar la aplicación Streamlit en segundo plano ──────────────────
!streamlit run 0_🔩_Simulacion_Sistemas.py &> /dev/null &

# ── Esperar unos segundos para que Streamlit esté en línea ────────────
time.sleep(3)

# ── Iniciar el túnel con Cloudflared ──────────────────────────────────
!cloudflared tunnel --url http://localhost:8501 > cloudflared.log 2>&1 &

# ── Dar tiempo para que Cloudflared genere la URL ─────────────────────
time.sleep(8)

# ── Leer el log para extraer la URL pública del túnel ──────────────────
url = None
with open("cloudflared.log") as file:
    for line in file:
        match = re.search(r"https://[a-z0-9-]+\.trycloudflare\.com", line)
        if match:
            url = match.group(0)
            break

# ── Mostrar resultado al usuario ───────────────────────────────────────
if url:
    print("✅ Tu dashboard está disponible en:\n", url)
else:
    print("⚠️ No se pudo obtener la URL. Revisa 'cloudflared.log' para más detalles.")


✅ Tu dashboard está disponible en:
 https://mh-deaf-makeup-holes.trycloudflare.com


## Conclusiones


- **Se comprendió en profundidad** el comportamiento de los sistemas de segundo orden, tanto en su forma mecánica (masa–resorte–amortiguador) como en su equivalente eléctrico (circuito R–L–C).  

- **Se exploró interactivamente** los diferentes regímenes de amortiguamiento (subamortiguado, críticamente amortiguado, sobreamortiguado e inestable) y observar cómo el factor ζ y la frecuencia natural ωₙ afectan la forma de la respuesta temporal y la posición de polos y ceros.  
- **Se visualizó de manera clara** los diagramas de Bode, así como las respuestas al impulso, al escalón y a la rampa, tanto en lazo abierto como en lazo cerrado, reforzando el entendimiento de conceptos clave como la ganancia de pico, el ancho de banda y la estabilidad.  
- **Se calcularon parámetros temporales** —tiempo de subida, sobreimpulso, tiempo al pico y tiempo de establecimiento—, y relacionarlos cuantitativamente con ζ y ωₙ para comprender cómo ajustar un sistema real según requisitos de rendimiento.  
  


